"""
this script does not work anymore, but gives you an idea of how to go about scraping a website
"""

In [8]:
# default modules
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Callable, Union
%matplotlib inline
import os
import shelve
import functools
import logging

10-02-2019 09:11:36 DEBUG    [__init__.py:90] backend module://ipykernel.pylab.backend_inline version unknown


In [10]:
# setting pandas display settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 500)

In [11]:
# setting up logger
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',
    datefmt = '%d-%m-%Y %H:%M:%S',
    level=logging.DEBUG,
    # filename='logs.txt'
)
logger = logging.getLogger(__name__)

In [12]:
# classes

class ShelveApi:
    """
    simple wrapper for loading and saving shelves
    """
    def __init__(self, filename):
        self.filename = filename
    
    def load_from_shelve(self, key):
        """
        load the value that corresponds to a certain key from a shelve
        """
        d = shelve.open(self.filename, 'c')
        val = d[key]
        d.close()
        return val
    
    def save_to_shelve(self, val, key):
        """
        save values to shelve that corresponds to a specified key
        """
        d = shelve.open(self.filename, 'c')
        d[key] = val
        d.close()
    
    def show_keys(self):
        """
        show all keys that exisit in the shelve
        """
        d = shelve.open(self.filename, 'c')
        print(list(d.keys()))
        d.close()

### scraping pcp

In [9]:
# webscraping/parsing modules
import requests
from bs4 import BeautifulSoup, SoupStrainer
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [22]:
# definitions
BASE_URL = 'https://www.some_url.co.uk' # not a real url

In [17]:
# instantiating shelve wrapper class
pcp_shelve = ShelveApi('pcp_shelve')

In [21]:
# using selenium to activate javascript and get html

driver = webdriver.Chrome() # opens a browser which python can automate, NB you need to download ChromeDriver.exe and have it in the same
# folder as the script for this to work
driver.get('https://www.some_url.co.uk/data_page') # load page

driver.find_element_by_css_selector("a[href*='java'][href*='All']").click() # using CSS selectors to click on the element of the page 
# where the href of the page element contains the text 'java' and 'All'
pcp_shelve.save_to_shelve(driver.page_source,'index_page_html') # grabbing the subsequent page content and saving it
driver.quit()

In [26]:
# retiving each plant page url from html

# this block of code uses beautiful soup to grab all the hrefs of the page we just saved
# it then uses a list comprehesion to filter be hrefs that contain the string 'PlantID'

def get_all_urls(soup) -> List:
    """
    return a list of all the hrefs included in an soup object
    """
    urls = []
    for link in soup.find_all('a'):
        if link.has_attr('href'):
            urls.append(link.get('href'))
    return urls

soup = BeautifulSoup(pages['plant_index'], 'html.parser')
plant_id_urls = [urljoin(BASE_URL, url_tail) for url_tail in get_all_urls(soup) if 'PlantID' in url_tail]
plant_id_urls[:5]

['http://www.peakcottageplants.co.uk/ViewPlantDetails.aspx?PlantID=4',
 'http://www.peakcottageplants.co.uk/ViewPlantDetails.aspx?PlantID=8',
 'http://www.peakcottageplants.co.uk/ViewPlantDetails.aspx?PlantID=9',
 'http://www.peakcottageplants.co.uk/ViewPlantDetails.aspx?PlantID=11',
 'http://www.peakcottageplants.co.uk/ViewPlantDetails.aspx?PlantID=15']

In [ ]:
# requesting html for each plant page in the list

def get_rsps(urls: List) -> List[requests.models.Response]:
    """
    return a list of request responses
    """
    return [requests.get(url) for url in urls]

plant_page_rsps = get_rsps(plant_id_urls)
pcp_shelve.save_to_shelve(plant_page_rsps, 'plant_page_rsps')

In [29]:
# plant page parser tools

def trim(string):
    """
    remove all escape characters and double white spaces from a string
    """
    return ' '.join(string.split())

# this class takes in an indivicual plant pages and uses beautful soup to grab
# the elements of the page we are interested it, this will always to specific to
# the page you are scraping

class PlantPageParser:
    """
    parse key information from plant page html into a dictionary
    """
    def __init__(self, html):
        self.soup = BeautifulSoup(html, 'html.parser')
    
    @property
    def get_table(self):
        return self.soup.find('table', {'style': 'width:390px'})
    
    @property
    def parse_table_rows(self) -> Dict:
        data = {}
        for tr in self.get_table.find_all('tr'):
            field, val = tr.find_all('td')
            data[trim(field.text)] = trim(val.text)
        return data
    
    @property
    def get_plant_nm(self) -> Dict:
        nm = self.soup.find('h2').text
        return {'Name:': trim(nm)}
    
    @property
    def get_parsed_page(self) -> Dict:
        return {**self.get_plant_nm, **self.parse_table_rows}

In [123]:
# parsing pages into a dataframe

data = []
for rsp in rsps:
    parse = PlantPageParser(rsp.text)
    data.append(parse.get_parsed_page)
df = pd.DataFrame(data)
df.head()

,Common Name:,Conditions:,Description:,Flower Colour:,Hardiness:,Height (cm):,In Stock?:,Leaf Colour:,Name:,Season of Interest:,Soil Type:
0,Yarrow,Full Sun,A vigorous plant with bright magenta-pink flow...,Pink,Fully Hardy,60,Yes,Green,Achillea millefolium - Cerise Queen,Summer,Moist but well-drained
1,Sneezewort,Full Sun,"This strong perennial bears tight, button-like...",White,Fully Hardy,60,Yes,Green,Achillea ptarmica - Benary's Pearl,Summer,Moist but well-drained
2,Not defined,Sun or part shade,Soft pink flowers in a large umbel throughout ...,Pink,Fully Hardy,60,Yes,Green,Achillea siberica - Love Parade,Summer,Moist but well-drained
3,Monkshood,Partial shade,White flowers that become tinged with rose as ...,White,Fully Hardy,90,Yes,Dark green,Aconitum bicolor - Mother of Pearl,Summer,Moist but well-drained
4,Wolf's Bane,Partial shade,"Panicles of pale yellow, hooded flowers are pr...",Yellow,Fully Hardy,100,Yes,Light green,Aconitum lycoctonum syn. Vulparia,Summer,Any


In [124]:
df.to_csv('pcp_plants.csv', index=False)